In [ ]:
import pandas as pd
import holoviews as hv
import numpy as np
import datetime
hv.extension('bokeh')
import re
import cudf
import time
import cupy
import numpy
from holoviews.operation.datashader import datashade,dynspread
from collections import Counter
import geoviews
from cartopy import crs
geoviews.extension("bokeh")
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')
# import lux
# from lux.core.frame import LuxDataFrame

In [ ]:
'''
The following code will utilize the csv containing the important graphs to be plotted, and plot them using HoloViz
'''

In [ ]:
# saved_interesting_plots.csv is the csv extracted from LUX by altering its code
dat = pd.read_csv("data/interesting/hpi_interesting.csv") #in pandas as few rows only
df = cudf.read_csv("data/hpi.csv")
df=df.dropna(axis=0)
df

In [ ]:
# Display the csvs data extracted from LUX
for lis in dat['collection']:
    print(lis[1:-1])

In [ ]:
#When fips code is used in the dataset rather than state name
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}

In [ ]:
s =time.time()
adder = False
flag = False
left_name="name"
grph_num = 1
for words in dat['collection']:
    lets = words.split("\n")
    for let in lets:
        res = re.findall(r'\w+', let)
        graph = res[-4]
        form = ""
        if graph =="histogram":
            starting = time.time()
            xlabel = res[3]
            ylabel = res[6]
            x = df[xlabel]
            x = cupy.fromDlpack(x.to_dlpack())
            frequencies, edges = cupy.histogram(x, bins=10)
            if len(frequencies)>10:
                lis = list(zip(frequencies,edges))
                lis.sort(key=lambda x:x[0],reverse=True)
                lis=lis[:10]
                frequencies, edges = zip(*lis)
                frequencies = cupy.array(frequencies)
                edges = cupy.array(frequencies)
            if abs(max(edges))>10000: form = '%.1e'
            curve=hv.Histogram((edges.get(), frequencies.get())).opts(axiswise=True,  xformatter=form, xlabel=xlabel, ylabel=ylabel, title = graph +" : "+str(grph_num))#yformatter='%.1e',
            grph_num+=1
            if not adder: adder = curve
            else: adder+=curve
            print("time in histogram :", time.time() -starting)
        elif graph =="bar":
            starting = time.time()
            xlabel ="Records"
            ylabel = res[5]
            x=df.groupby(ylabel).count()
            x.reset_index(inplace=True) 
            lis = list(zip(x[ylabel].values_host, x.iloc[:, 1].values_host))
            if len(lis)>10: 
                lis.sort(key=lambda x:x[1],reverse=True)
                lis=lis[:10]
            #print(lis)
            if abs(x.iloc[:, 1].max())>10000: form = '%.1e'
            curve = hv.Bars(lis).opts(invert_axes=True).opts(axiswise=True, xlabel=ylabel, ylabel =xlabel,xformatter=form, title = graph +" : "+str(grph_num))
            grph_num+=1
            if not adder: adder = curve
            else: adder+=curve
            print("time in inverted bar :", time.time() -starting)
        elif graph =="line":
            starting = time.time()
            xlabel = res[2]
            factor = re.search(xlabel + '(.*)y', "".join(res)).group(1)
            ylabel = "Records"
            if factor == 'dayofweek':
                x = cudf.DatetimeIndex(df[xlabel])
                x = cudf.DataFrame(x.dayofweek, columns = ["date"])
            elif factor =='month':
                x = cudf.DatetimeIndex(df[xlabel])
                x = cudf.DataFrame(x.month, columns = ["date"])
            elif factor =="year":
                x = cudf.DatetimeIndex(df[xlabel])
                x = cudf.DataFrame(x.year, columns = ["date"])
            else:
                x = df.rename(columns={xlabel: "date"})
            x["count"] = x['date']
            x = x.groupby("date").count()
            x.reset_index(inplace=True)
            x = x.sort_values('date')
            lis = list(zip(x['date'].values_host, x['count'].values_host))
            curve = hv.Curve(lis, hv.Dimension(xlabel), "Number of " +ylabel).opts(axiswise=True,yformatter='%.1e', xlabel=xlabel, ylabel=ylabel , title = graph +" : "+str(grph_num))
            grph_num+=1
            if not adder: adder = curve
            else: adder+=curve
            print("time in linecurve :", time.time() -starting)
        elif graph == "scatter":
            starting = time.time()
            xlabel = res[2]
            ylabel = res[4]
            x = df[xlabel]
            y = df[ylabel]
            z = cudf.concat([x,y],axis=1)
            if abs(x.max())>10000: form = '%.1e'
            curve = dynspread(datashade(hv.Scatter(z)).opts(axiswise=True, xformatter = form, xlabel=xlabel, ylabel=ylabel, title = graph +" : "+str(grph_num)),threshold=0.75)#yformatter='%.1e', 
            grph_num+=1
            if adder==False: adder = curve
            else: adder+=curve
            print("time in scatterplot :", time.time() -starting)
        elif graph =="geographical":
            starting = time.time()
            geo = res[2]
            vals = res[5]
            x=df.groupby(geo).mean()
            x.reset_index(inplace=True)
            if not flag:
                if geo in ["states","state","States","State", "STATES", "STATE"]:
                    geography = gpd.read_file("us-states.json")
                    if isinstance(x[geo].iloc[0],numpy.int64):
                        left_name = "fips_num"
                        geography[left_name] = geography["id"].apply(lambda x: int(state_codes[x]))
                    geography_pop = geography.merge(x.to_pandas(), left_on=left_name, right_on=geo)
                elif geo in ["Country", "COUNTRY", "country", "COUNTRIES","countries", "Countries"]:
                    geography = gpd.read_file("countries.geojson") 
                    geography_pop = geography.merge(x.to_pandas(), left_on="ADMIN", right_on=geo)
                flag =True
            if geo in ["states","state","States","State", "STATES", "STATE"]:
                curve = geoviews.Polygons(data=geography_pop, vdims=[vals, geo]).opts(xlim=(-170, -60), ylim=(10,75), projection = crs.LambertCylindrical(), height=300, width=400, title=vals+" : "+str(grph_num), tools=["hover", ], colorbar=True, colorbar_position="right")
            elif geo in ["Country", "COUNTRY", "country", "COUNTRIES","countries", "Countries"]:
                curve = geoviews.Polygons(data=geography_pop, vdims=[vals, geo]).opts( height=300, width=400, title=vals+" : "+str(grph_num), tools=["hover", ], colorbar=True, colorbar_position="right")
            grph_num+=1
            if adder==False: adder = curve
            else: adder+=curve
            print("time in choropleth :", time.time() -starting)

print("total time : ", time.time()-s)

In [ ]:
start=time.time()

In [ ]:
hv.Layout(adder)

In [ ]:
print(time.time()-start)